In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests
import json
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
API_URL = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action"

In [3]:
#Function to retrieve data from Toronto Open Data API

def get_package_tables(package_id_or_name):
    params = { "id": package_id_or_name}
    package = requests.get(f"{API_URL}/package_show", params).json()
    tables=[]
    
    for idx, resource in enumerate(package["result"]["resources"]):
        if resource["datastore_active"]:
            url = f"{API_URL}/datastore_search"
            p = {"id": resource["id"]}
            total_record_count = requests.get(url, {"id": resource["id"] }).json()["result"]["total"]
            p["limit"] = total_record_count
            
            data = requests.get(url, p).json()
            df = pd.DataFrame(data["result"]["records"])
            tables.append(df)

    return tables

In [4]:
development_applications = get_package_tables("development-applications")
apps = development_applications[0]
apps

,_id,APPLICATION#,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,HEARING_DATE,POSTAL,REFERENCE_FILE#,STATUS,STREET_DIRECTION,STREET_NAME,STREET_NUM,STREET_TYPE,X,Y,APPLICATION_NUMBER,REFERENCE_FILE_NUMBER
0,27337555,None,MV,2017-03-28,To construct a new detached dwelling with an i...,2017-07-13,M6M,None,Approved with Conditions,,CRAYDON,37,AVE,305020.004,4838763.402,17 135434 WET 11 MV,A0295/17EYK
1,27337556,None,CO,2017-05-04,To obtain consent for a long term lease of 41 ...,2017-07-26,M5T,None,Closed,,COLLEGE,250,ST,312921.265,4835186.308,17 154131 STE 20 CO,B0042/17TEY
2,27337557,None,MV,2017-05-02,To maintain a rear detached garage with an att...,2017-06-15,M6L,None,Closed,,WICKFORD,12,DR,306384.318,4840977.851,17 152745 WET 12 MV,A0402/17EYK
3,27337558,None,MV,2017-04-07,To construct a new two-storey dwelling with in...,2018-01-11,M2N,None,Closed,,STUART,69,CRES,311813.762,4845855.716,17 141261 NNY 23 MV,A0364/17NY
4,27337559,None,MV,2017-04-24,To alter the existing two-storey semi-detached...,2017-08-23,M4M,None,Closed,,PAPE,239,AVE,317768.394,4835992.594,17 148318 STE 30 MV,A0472/17TEY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75565,27337550,None,MV,2017-07-10,To convert the existing two-storey semi-detach...,2017-11-08,M6H,None,Closed,,ST CLARENS,600,AVE,309286.488,4835718.975,17 195407 STE 18 MV,A0755/17TEY
75566,27337551,None,MV,2017-03-08,To alter the existing two-storey semi-detached...,2017-07-12,M4L,None,Closed,,ALTON,85,AVE,318609.955,4836029.213,17 126052 STE 30 MV,A0279/17TEY
75567,27337552,None,MV,2017-03-30,To construct a new two-storey detached dwellin...,2017-08-09,M4J,None,Closed,,DUSTAN,7,CRES,317771.974,4839386.367,17 136691 STE 29 MV,A0374/17TEY
75568,27337553,None,MV,2017-06-16,The applicant is seeking relief from the provi...,2017-09-07,M1K,None,Closed,,LINDEN,122,AVE,324283.178,4842281.594,17 181571 ESC 35 MV,A0221/17SC


In [ ]:
#Remove speacial charachters from columns

apps.columns = apps.columns.str.replace('[#,@,&]','')

In [ ]:
#Drop unnecessary columns

cols_to_drop = ['_id','APPLICATION','HEARING_DATE','REFERENCE_FILE','X','Y','REFERENCE_FILE_NUMBER']
apps.drop(columns=cols_to_drop, inplace = True)

In [ ]:
#Convert application date to datetime

apps['DATE_SUBMITTED'] = pd.to_datetime(apps['DATE_SUBMITTED'])
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74843 entries, 0 to 74842
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   APPLICATION_TYPE    74843 non-null  object        
 1   DATE_SUBMITTED      74843 non-null  datetime64[ns]
 2   DESCRIPTION         73555 non-null  object        
 3   POSTAL              74843 non-null  object        
 4   STATUS              74843 non-null  object        
 5   STREET_DIRECTION    74843 non-null  object        
 6   STREET_NAME         74824 non-null  object        
 7   STREET_NUM          74824 non-null  object        
 8   STREET_TYPE         74843 non-null  object        
 9   APPLICATION_NUMBER  74843 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 5.7+ MB


In [ ]:
#Set the min date to the date the TLAB came into effect
apps = apps[apps['DATE_SUBMITTED'] >= '2017-05-03']

In [ ]:
#Set address columns to title so you can later geoecode

cols_to_capitalize = ['STREET_DIRECTION','STREET_NAME','STREET_TYPE']

for i in cols_to_capitalize:
    apps[i] = apps[i].str.title()


In [ ]:
#Combine address columns to make final address

cols_to_combine = ['STREET_NUM','STREET_NAME','STREET_TYPE','STREET_DIRECTION']

apps["ADDRESS"] = apps[cols_to_combine].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

#Strip the white space at the end of each string and add Toronto for geocoding

apps["ADDRESS"] = apps["ADDRESS"].str.rstrip() +', Toronto'

#Drop unnecessary columns after the combine
drop_address = ['STREET_NUM','STREET_NAME','STREET_TYPE','STREET_DIRECTION']
apps.drop(columns=drop_address, inplace = True)

In [ ]:
apps.head()

,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS
0,MV,2019-05-27,To alter the existing two 2 ½-storey semi-deta...,M6J,Closed,19 160548 STE 10 MV,"943 Dundas St W, Toronto"
1,MV,2019-05-27,The applicant is seeking relief from the provi...,M1E,Closed,19 160556 ESC 24 MV,"59 Cumber Ave, Toronto"
2,MV,2019-05-27,To alter the existing two-storey semi-detached...,M6H,Closed,19 160557 STE 09 MV,"383 Margueretta St, Toronto"
3,OZ,2019-08-12,The applicant is proposing a 9-storey mixed-us...,M8Y,Council Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto"
4,MV,2019-08-12,"To permit the construction of a new roof, in c...",M5N,Closed,19 204540 NNY 08 MV,"3 Glen Willow Pl, Toronto"


In [ ]:
apps.APPLICATION_TYPE.unique()

array(['MV', 'OZ', 'CO', 'SA', 'TLAB', 'CD', 'SB', 'PL'], dtype=object)

In [ ]:
#Remove PL and TLAB. PL stands for part lot and are typically always accepted. Remove TLAB because it doesn't make sense
#to make a prediction on applications where the result is always denied

apps.drop(apps[apps.APPLICATION_TYPE == 'TLAB'].index, inplace=True)
apps.drop(apps[apps.APPLICATION_TYPE == 'PL'].index, inplace=True)

In [ ]:
apps.APPLICATION_TYPE.value_counts()

MV    16419
SA     2524
OZ     2397
CO     1938
CD     1502
SB      637
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
#Rename the application types from their acronyms

apps['APPLICATION_TYPE'] = apps['APPLICATION_TYPE'].replace({'MV':'Minor Variance',
                                                             'CD':'Condominium',
                                                             'OZ':'Official Plan Rezoning',
                                                             'SA':'Site Plan Application',
                                                             'SB':'Subdivision',
                                                             'CO':'Consent'})
apps.APPLICATION_TYPE.value_counts()

Minor Variance            16419
Site Plan Application      2524
Official Plan Rezoning     2397
Consent                    1938
Condominium                1502
Subdivision                 637
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
#Check the value counts of status to see all possibilities that need to be renamed
apps.STATUS.value_counts()

Closed                      15828
Under Review                 4410
Accepted                      855
TLAB Appeal                   503
Hearing Scheduled             481
OMB Appeal                    401
Draft Plan Approved           374
Application Withdrawn         333
NOAC Issued                   264
Postponed                     264
Conditional Consent           261
Deferred                      239
Application Received          233
Council Approved              191
Tentatively Scheduled         160
Appeal Received               133
Approved with Conditions      113
Final Approval Completed      100
Approved                       76
In Progress                    76
OMB Approved                   37
Refused                        28
Await Expiry Date              23
In Process                     12
Under Review                    8
Prepare Notice                  4
Notice Prepared                 4
Hearing Rescheduled             3
Notification Completed          2
Appeal Withdra

In [ ]:
#Assign remaining applications to either appealed or approved based on their status. If the status is not found set for removal

denied_keywords = ['OMB Appeal','TLAB Appeal','OMB Approved','Appeal Received','Appeal Decision Pending',
                   'Appeal Received by TLAB','Appeal Received by C of A','Appeal Dismissed','Appeal Withdrawn','Refused']

approved_keywords = ['Accepted','Draft Plan Approved','Council Approved','Approved','NOAC Issued','Approved with Conditions',
                     'Conditional Consent','Final Approval Completed']

apps['STATUS'] = apps.STATUS.apply(lambda x: 'Denied' if x in denied_keywords else 'Approved' if x in approved_keywords else 'Remove')

apps.STATUS.value_counts()

Remove      22080
Approved     2234
Denied       1103
Name: STATUS, dtype: int64

In [ ]:
#Remove rows that are assigned for removal

apps.drop(apps[apps.STATUS == 'Remove'].index, inplace=True)

In [ ]:
apps

,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS
3,Official Plan Rezoning,2019-08-12,The applicant is proposing a 9-storey mixed-us...,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto"
8,Site Plan Application,2019-08-09,Site Plan Approval application to construct a ...,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto"
17,Consent,2019-05-07,To obtain consent to sever the property into t...,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto"
21,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5868-5870 Yonge St, Toronto"
22,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5840 Yonge St, Toronto"
...,...,...,...,...,...,...,...
74764,Site Plan Application,2019-05-13,Site Plan Control application to permit a prop...,,Approved,19 153681 WET 03 SA,"2 Wickman Rd, Toronto"
74774,Site Plan Application,2019-09-03,Proposed restaurant with a drive-through.,M9L,Approved,19 215475 WET 07 SA,"4915 Steeles Ave W, Toronto"
74790,Minor Variance,2018-12-10,To construct a new dwelling.,M2P,Denied,18 266685 NNY 15 MV,"14 Brookfield Rd, Toronto"
74818,Official Plan Rezoning,2018-12-21,Official Plan and Zoning By-law Amendment to p...,,Approved,18 271373 STE 13 OZ,"202 Jarvis St, Toronto"


In [ ]:
apps.STATUS.value_counts()

Approved    2234
Denied      1103
Name: STATUS, dtype: int64

In [ ]:
apps.groupby(['APPLICATION_TYPE'])['STATUS'].value_counts()

APPLICATION_TYPE        STATUS  
Condominium             Approved    380
Consent                 Approved    530
                        Denied       89
Minor Variance          Approved    775
                        Denied      449
Official Plan Rezoning  Denied      505
                        Approved    191
Site Plan Application   Approved    354
                        Denied       50
Subdivision             Denied       10
                        Approved      4
Name: STATUS, dtype: int64

In [ ]:
#Remove condo applications because they are all approved and subdivision because there are very few

apps.drop(apps[apps.APPLICATION_TYPE == 'Condominium'].index, inplace=True)
apps.drop(apps[apps.APPLICATION_TYPE == 'Subdivision'].index, inplace=True)

In [ ]:
apps.head()

,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS
3,Official Plan Rezoning,2019-08-12,The applicant is proposing a 9-storey mixed-us...,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto"
8,Site Plan Application,2019-08-09,Site Plan Approval application to construct a ...,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto"
17,Consent,2019-05-07,To obtain consent to sever the property into t...,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto"
21,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5868-5870 Yonge St, Toronto"
22,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5840 Yonge St, Toronto"


## Combine Application Numbers To Get The Number of Properties

If you don't combine the number of properties by the application number you will have duplicates. Two different properties might appear as different applications, because the addresses are different, but in fact they belong to the same application.

In [ ]:
apps_num_prop = apps[['DATE_SUBMITTED','APPLICATION_NUMBER','ADDRESS']]
apps_num_prop

,DATE_SUBMITTED,APPLICATION_NUMBER,ADDRESS
3,2019-08-12,19 204533 WET 03 OZ,"689 The Queensway, Toronto"
8,2019-08-09,19 203460 STE 11 SA,"321 Davenport Rd, Toronto"
17,2019-05-07,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto"
21,2019-09-05,19 216670 NNY 18 SA,"5868-5870 Yonge St, Toronto"
22,2019-09-05,19 216670 NNY 18 SA,"5840 Yonge St, Toronto"
...,...,...,...
74764,2019-05-13,19 153681 WET 03 SA,"2 Wickman Rd, Toronto"
74774,2019-09-03,19 215475 WET 07 SA,"4915 Steeles Ave W, Toronto"
74790,2018-12-10,18 266685 NNY 15 MV,"14 Brookfield Rd, Toronto"
74818,2018-12-21,18 271373 STE 13 OZ,"202 Jarvis St, Toronto"


In [ ]:
count_series = apps_num_prop.groupby(['APPLICATION_NUMBER','DATE_SUBMITTED']).size()
apps_num_prop = count_series.to_frame(name = 'Number_of_Properties').reset_index()
apps_num_prop

,APPLICATION_NUMBER,DATE_SUBMITTED,Number_of_Properties
0,17 154407 NNY 16 MV,2017-05-04,1
1,17 154800 STE 31 CO,2017-05-05,1
2,17 154850 STE 31 CO,2017-05-05,1
3,17 154966 NNY 16 CO,2017-05-05,1
4,17 154968 NNY 16 MV,2017-05-05,1
...,...,...,...
2009,22 100240 WET 02 MV,2022-01-03,1
2010,22 100318 STE 09 MV,2022-01-04,1
2011,22 100331 NNY 08 MV,2022-01-04,1
2012,22 100414 WET 05 MV,2022-01-04,1


In [ ]:
#Check if there are duplicates in application number after getting the property count

boolean = apps_num_prop['APPLICATION_NUMBER'].duplicated().any()
boolean

False

In [ ]:
#Combine the new data frame with number of properties to the original

apps_with_num_prop = pd.merge(apps, apps_num_prop, on= ['APPLICATION_NUMBER','DATE_SUBMITTED'], how = 'left')
apps_with_num_prop

,APPLICATION_TYPE,DATE_SUBMITTED,DESCRIPTION,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS,Number_of_Properties
0,Official Plan Rezoning,2019-08-12,The applicant is proposing a 9-storey mixed-us...,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto",1
1,Site Plan Application,2019-08-09,Site Plan Approval application to construct a ...,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto",1
2,Consent,2019-05-07,To obtain consent to sever the property into t...,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto",1
3,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5868-5870 Yonge St, Toronto",4
4,Site Plan Application,2019-09-05,The Local Planning Appeal Tribunal (LPAT) deci...,M2M,Denied,19 216670 NNY 18 SA,"5840 Yonge St, Toronto",4
...,...,...,...,...,...,...,...,...
2938,Site Plan Application,2019-05-13,Site Plan Control application to permit a prop...,,Approved,19 153681 WET 03 SA,"2 Wickman Rd, Toronto",1
2939,Site Plan Application,2019-09-03,Proposed restaurant with a drive-through.,M9L,Approved,19 215475 WET 07 SA,"4915 Steeles Ave W, Toronto",1
2940,Minor Variance,2018-12-10,To construct a new dwelling.,M2P,Denied,18 266685 NNY 15 MV,"14 Brookfield Rd, Toronto",1
2941,Official Plan Rezoning,2018-12-21,Official Plan and Zoning By-law Amendment to p...,,Approved,18 271373 STE 13 OZ,"202 Jarvis St, Toronto",2


## Binary Encode Each Application Based on Address

It is pretty common for applications to be grouped together. For example, submitting a minor variance and consent application at the same time. Official plan and site plan applications typically get submitted sequentially. It makes sense to binary encode these based on the address, as it removes duplicates, while showing that one property can have multiple applications on it.

In [ ]:
apps_to_binary_encode = apps[['ADDRESS','APPLICATION_TYPE']]

#Drop duplicates to ensure each address is unique

apps_to_binary_encode.drop_duplicates(inplace=True)

In [ ]:
apps_to_binary_encode.head()

,ADDRESS,APPLICATION_TYPE
3,"689 The Queensway, Toronto",Official Plan Rezoning
8,"321 Davenport Rd, Toronto",Site Plan Application
17,"21 Lorraine Gdns, Toronto",Consent
21,"5868-5870 Yonge St, Toronto",Site Plan Application
22,"5840 Yonge St, Toronto",Site Plan Application


In [ ]:
#Binary encode by grouping by address

final_binary_encode=apps_to_binary_encode.groupby('ADDRESS',sort=False).agg(list)

#Get dummies based on application type
final_binary_encode = final_binary_encode.assign(**final_binary_encode['APPLICATION_TYPE'].str.join('|').str.get_dummies()).reset_index()

In [ ]:
final_binary_encode.drop(columns=['APPLICATION_TYPE'], inplace = True)

In [ ]:
final_binary_encode.head(20)

,ADDRESS,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application
0,"689 The Queensway, Toronto",0,0,1,0
1,"321 Davenport Rd, Toronto",0,0,0,1
2,"21 Lorraine Gdns, Toronto",1,0,0,0
3,"5868-5870 Yonge St, Toronto",1,0,0,1
4,"5840 Yonge St, Toronto",0,0,0,1
5,"5858 Yonge St, Toronto",0,0,0,1
6,"4 Alvarado Pl, Toronto",0,1,0,0
7,"84 Queen'S Park, Toronto",0,0,1,1
8,"78 Queen'S Park, Toronto",0,0,1,1
9,"844 Don Mills Rd, Toronto",0,0,0,1


## Combine Number of Properties and Binary Encoded Dataframes 

Now that I have the number of properties that are tied to each application number, and the application types binary encoded based on address, I can merge all of these into a master data frame and remove the duplicates. This will leave me with each row being a distinct address (which can represent multiple properties) with all of the application types that were applied for.

In [ ]:
combined = pd.merge(apps_with_num_prop, final_binary_encode, on= 'ADDRESS', how = 'left')

#Drop columns that are no longer required
cols_to_drop = ['APPLICATION_TYPE','DESCRIPTION']

combined.drop(columns=cols_to_drop, inplace = True)

#Remove final duplicates
combined.drop_duplicates(inplace=True)

combined.head()

,DATE_SUBMITTED,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application
0,2019-08-12,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto",1,0,0,1,0
1,2019-08-09,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto",1,0,0,0,1
2,2019-05-07,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto",1,1,0,0,0
3,2019-09-05,M2M,Denied,19 216670 NNY 18 SA,"5868-5870 Yonge St, Toronto",4,1,0,0,1
4,2019-09-05,M2M,Denied,19 216670 NNY 18 SA,"5840 Yonge St, Toronto",4,0,0,0,1


In [ ]:
#Drop duplicates based on address and then application number
df =  combined.drop_duplicates(subset='ADDRESS', keep="last")
df =  df.drop_duplicates(subset='APPLICATION_NUMBER', keep="last")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1568 entries, 0 to 2942
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   DATE_SUBMITTED          1568 non-null   datetime64[ns]
 1   POSTAL                  1568 non-null   object        
 2   STATUS                  1568 non-null   object        
 3   APPLICATION_NUMBER      1568 non-null   object        
 4   ADDRESS                 1568 non-null   object        
 5   Number_of_Properties    1568 non-null   int64         
 6   Consent                 1568 non-null   int64         
 7   Minor Variance          1568 non-null   int64         
 8   Official Plan Rezoning  1568 non-null   int64         
 9   Site Plan Application   1568 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(4)
memory usage: 134.8+ KB


In [ ]:
df.head(5)

,DATE_SUBMITTED,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application
0,2019-08-12,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto",1,0,0,1,0
1,2019-08-09,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto",1,0,0,0,1
2,2019-05-07,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto",1,1,0,0,0
6,2018-10-15,M3A,Approved,18 242821 NNY 34 MV,"4 Alvarado Pl, Toronto",1,0,1,0,0
9,2019-08-16,M3C,Approved,19 206988 NNY 16 SA,"844 Don Mills Rd, Toronto",1,0,0,0,1


## Geocoding Using Nominatim

The original X and Y coordinates from the open data portal did not match the standard format latitude and longitude orientation. Using Nominatim I can geocode a location based on address and then extract the latitude and latidue for each location.

In [ ]:
locator = Nominatim(user_agent='TorontoDevApps')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['Location'] = df['ADDRESS'].apply(geocode)
df

,DATE_SUBMITTED,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application,Location
0,2019-08-12,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto",1,0,0,1,0,"(689, The Queensway, Etobicoke, Toronto, Golde..."
1,2019-08-09,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto",1,0,0,0,1,"(321, Davenport Road, The Annex, University—Ro..."
2,2019-05-07,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto",1,1,0,0,0,"(21, Lorraine Gardens, Islington, Etobicoke, T..."
6,2018-10-15,M3A,Approved,18 242821 NNY 34 MV,"4 Alvarado Pl, Toronto",1,0,1,0,0,"(Alvarado Place, Parkway West, Don Valley East..."
9,2019-08-16,M3C,Approved,19 206988 NNY 16 SA,"844 Don Mills Rd, Toronto",1,0,0,0,1,"(Don Mills Road, Don Mills, Don Valley East, N..."
...,...,...,...,...,...,...,...,...,...,...,...
2937,2018-12-21,M4B,Approved,18 271310 STE 19 OZ,"968 O'Connor Dr, Toronto",1,0,0,1,0,"(968, O'Connor Drive, Woodbine Gardens, Beache..."
2938,2019-05-13,,Approved,19 153681 WET 03 SA,"2 Wickman Rd, Toronto",1,0,0,0,1,"(2, Wickman Road, Etobicoke, Toronto, Golden H..."
2939,2019-09-03,M9L,Approved,19 215475 WET 07 SA,"4915 Steeles Ave W, Toronto",1,0,0,0,1,"(4915, Steeles Avenue West, Humber River—Black..."
2940,2018-12-10,M2P,Denied,18 266685 NNY 15 MV,"14 Brookfield Rd, Toronto",1,0,1,0,0,"(14, Brookfield Road, Hogg's Hollow, Don Valle..."


In [ ]:
#Find out how many properties did not get geocoded

df['Location'].isna().sum()

23

In [ ]:
#Drop the properties that did not get geocoded from the data frame

df.dropna(inplace=True)

In [ ]:
#Create lat and long columns after geocoding

df['Latitude']=df['Location'].apply(lambda x: x.latitude if x != None else None)
df['Longitude']=df['Location'].apply(lambda x: x.longitude if x != None else None)

In [ ]:
df.head()

,DATE_SUBMITTED,POSTAL,STATUS,APPLICATION_NUMBER,ADDRESS,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application,Location,Latitude,Longitude
0,2019-08-12,M8Y,Approved,19 204533 WET 03 OZ,"689 The Queensway, Toronto",1,0,0,1,0,"(689, The Queensway, Etobicoke, Toronto, Golde...",43.626854,-79.499656
1,2019-08-09,M5R,Approved,19 203460 STE 11 SA,"321 Davenport Rd, Toronto",1,0,0,0,1,"(321, Davenport Road, The Annex, University—Ro...",43.674647,-79.400973
2,2019-05-07,M9B,Approved,19 150177 WET 02 CO,"21 Lorraine Gdns, Toronto",1,1,0,0,0,"(21, Lorraine Gardens, Islington, Etobicoke, T...",43.652096,-79.547083
6,2018-10-15,M3A,Approved,18 242821 NNY 34 MV,"4 Alvarado Pl, Toronto",1,0,1,0,0,"(Alvarado Place, Parkway West, Don Valley East...",43.753666,-79.338064
9,2019-08-16,M3C,Approved,19 206988 NNY 16 SA,"844 Don Mills Rd, Toronto",1,0,0,0,1,"(Don Mills Road, Don Mills, Don Valley East, N...",43.735174,-79.342838


In [ ]:
#Drop columns that are no longer needed and can't be used in the prediction model

final_cols_to_drop = ['DATE_SUBMITTED','POSTAL','APPLICATION_NUMBER','ADDRESS','Location']

df.drop(columns=final_cols_to_drop, inplace = True)

In [ ]:
df.head()

,STATUS,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application,Latitude,Longitude
0,Approved,1,0,0,1,0,43.626854,-79.499656
1,Approved,1,0,0,0,1,43.674647,-79.400973
2,Approved,1,1,0,0,0,43.652096,-79.547083
6,Approved,1,0,1,0,0,43.753666,-79.338064
9,Approved,1,0,0,0,1,43.735174,-79.342838


In [ ]:
#Rename the status column

df.rename(columns={'STATUS': 'Status'}, inplace=True)

In [ ]:
#Sort the valus to see if any are outside of Toronto

df.sort_values(by=['Longitude'], ascending = False)

,Status,Number_of_Properties,Consent,Minor Variance,Official Plan Rezoning,Site Plan Application,Latitude,Longitude
2712,Approved,1,1,0,0,0,46.452468,-63.379963
1714,Approved,1,1,0,0,0,46.452468,-63.379963
2245,Approved,1,1,0,0,0,38.909465,-77.045140
2639,Approved,3,1,0,0,0,43.789452,-79.130461
1356,Approved,1,1,1,0,0,43.788621,-79.136475
...,...,...,...,...,...,...,...,...
1863,Approved,1,0,0,0,1,43.715209,-79.604134
1170,Denied,1,0,1,0,0,43.743618,-79.604611
878,Approved,1,0,0,0,1,43.735782,-79.613155
1529,Approved,1,0,0,0,1,43.745399,-79.614545


In [ ]:
#Remove points that are outside of Toronto

df = df[df.Longitude <= -79.130461]

In [ ]:
#Export to excel

df.to_excel("geocode_output.xlsx")